In [ ]:
import delearn as dl
import torch as tt
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

dtype=tt.float32

In [ ]:
from mlp import MLP_PARAMS_DICT

parameters_dict = MLP_PARAMS_DICT(
    input_size=1,
    layer_sizes=[64,64,64],
    output_size=1,
    dtype=dtype,
)
parameters_dict.keys()

In [ ]:

module = dl.Module(parameters_dict)
dl.rand_parameters(module)
def forward(m, x): 
    return ( tt.matmul(
        tt.relu( tt.matmul(
        tt.tanh( tt.matmul(
        tt.sigmoid( tt.matmul(
        x
        , m['w0']) + m['b0'] )
        , m['w1']) + m['b1'] )
        , m['w2']) + m['b2'] )
        , m['w3']) + m['b3'] )

dl.show_parameters(module)


In [ ]:
# dl.show_parameters(module)
# dl.count_parameters(module)
# for i,(n,p) in module: print(i, n, p.shape)
# dl.show_parameters(module, values=True)

# Define Forward function

In [ ]:
tx = tt.linspace(-25, 25, 500, dtype=dtype)
ty = tt.sin(tx)
tds = dl.Task( tx.unsqueeze(-1), ty.unsqueeze(-1), )
print(len(tds))

txx, tyy = next(iter(DataLoader(tds, batch_size=len(tds), shuffle=False)))
plt.scatter(txx,tyy, color='black', marker='.')

In [ ]:
vx = tt.linspace(-25, 25, 500, dtype=dtype)
vy = tt.sin(vx)
vds = dl.Task( vx.unsqueeze(-1), vy.unsqueeze(-1), )
print(len(vds))

vxx, vyy = next(iter(DataLoader(vds, batch_size=len(vds), shuffle=False)))
plt.scatter(vxx,vyy, color='black', marker='.')

In [ ]:
lossf = tt.nn.MSELoss()

In [ ]:

tloss, tpp = dl.Trainer.predict_batch(txx, tyy, forward, module, lossf)
vloss, vpp = dl.Trainer.predict_batch(vxx, vyy, forward, module, lossf)

plt.scatter(txx,tyy, color='black', marker='.')
plt.scatter(vxx,vyy, color='black', marker='.')
plt.scatter(txx,tpp, color='red', marker='.')
plt.scatter(vxx,vpp, color='blue', marker='.')
plt.show()
print(tloss, vloss)

In [ ]:
xx, yy = txx, tyy
loss, pp = dl.Trainer.predict_batch(xx, yy, forward, module, lossf)

plt.scatter(xx,yy, color='black', marker='.')
plt.scatter(xx,pp, color='red', marker='.')
plt.show()
print(loss)

In [ ]:
callback = dl.TrainingCallback(val_data=(vxx,vyy), forward=forward, lossf=lossf)

In [ ]:
optimf = dl.SGD( lr=0.025 )

In [ ]:
optimf = dl.SGD( lr=0.025,momentum=0.02, dampening=0.05, weight_decay=0.01, nesterov=True )

In [ ]:
callback.clear()

In [ ]:

dl.Trainer.train_dataset(
    forward=forward,
    module=module,
    lossf=lossf,
    optimf=optimf,
    dataf = lambda e: DataLoader(tds, batch_size=10, shuffle=True, drop_last=False),
    n=2000,
    batch_mode=False,
    create_graph=False,
    callback=callback,
)
callback.plot_results()

In [ ]:

tloss, tpp = dl.Trainer.predict_batch(txx, tyy, forward, module, lossf)
vloss, vpp = dl.Trainer.predict_batch(vxx, vyy, forward, module, lossf)

plt.scatter(txx,tyy, color='black', marker='.')
plt.scatter(vxx,vyy, color='black', marker='.')
plt.scatter(txx,tpp, color='red', marker='.')
plt.scatter(vxx,vpp, color='blue', marker='.')
plt.show()
print(tloss, vloss)

In [ ]:
loss, pp = dl.Trainer.predict_batch(xx, yy, forward, module, lossf)

plt.scatter(xx,yy, color='black', marker='.')
plt.scatter(xx,pp, color='red', marker='.')
print(loss)

In [ ]:
plt.plot(callback.val_losses)